# Machine Translation with Sequence-to-Sequence and RNNs

This lab will demonstrate fundamentals of Sequence-to-sequence RNN model for translation.  Using various libraries: `pandas`, `numpy`, `sklearn`, `seaborn`, `matplotlib`, `torch`, `spacy`.  (NOTE: we will use `spacy` for the natural language processing (NLP). 

Will demonstrate use of the Multi30K dataset (a large machine translation dataset with extensive English to German sentence pairs). 


In [22]:
import os
import sys
import warnings

warnings.filterwarnings('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'

# Install packages
os.system(f"{sys.executable} -m pip install -qq 'numpy<2.0' 2>/dev/null")
os.system(f"{sys.executable} -m pip install -qq torch==2.2.2 torchvision==0.17.2 torchtext==0.17.2 2>/dev/null")
os.system(f"{sys.executable} -m pip install -qq 'spacy<3.8' 'thinc<8.3' 2>/dev/null")
os.system(f"{sys.executable} -m pip install -qq pandas matplotlib seaborn scikit-learn portalocker 2>/dev/null")
os.system(f"{sys.executable} -m pip install -qq torchdata==0.7.1 nltk 2>/dev/null")
os.system(f"{sys.executable} -m spacy download en_core_web_sm 2>/dev/null")

os.system(f"{sys.executable} -m spacy download de_core_news_sm 2>/dev/null")

# Verify
print("Installation complete! Testing imports...")
import torch
import spacy
import numpy as np
print(f"✓ NumPy {np.__version__}")
print(f"✓ PyTorch {torch.__version__}")
print(f"✓ spaCy {spacy.__version__}")

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.2 MB/s  0:00:00m0:00:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 29.7 MB/s  0:00:00 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
Installation complete! Testing imports...
✓ NumPy 1.26.4
✓ PyTorch 2.2.2
✓ spaCy 3.7.5


## Import required libraries

In [26]:
from torchtext.data.utils import get_tokenizer #import get_tokenizer function from torchtext
from torchtext.vocab import build_vocab_from_iterator #import function that builds vocabulary from tokenized text
from nltk.translate.bleu_score import sentence_bleu #Used for evals/BLEU score metrics
from torchtext.datasets import multi30k, Multi30k #Import the utilities/functions to access Multi30k dataset
from typing import Iterable, List # For type hints
from torch.nn.utils.rnn import pad_sequence #for batch sequence length noramlization
from torch.utils.data import DataLoader #for creation of data batches
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext #Torchtext for NLP tasks

import torch #Main PyTorch library for tensor operations
import torch.nn as nn #utilities for building blocks of the neural network (e.g. layers, loss functions)
import torch.optim as optim #optimization algo for model weight updates during training

#suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# Sequence to Sequence Model

Sequence-to-sequence or `seq2seq` models useful for: 
- Translation from source language to target language
- Chat bots (e.g. answering question or generatugin natural language responses to input sequences)
- Summarization